In [1]:
# import pyscfad
# from pyscfad import gto,dft,scf
import pyscf
from pyscf import gto,dft,scf

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
import numpy as np
import jax.numpy as jnp
import scipy
from ase import Atoms
from ase.io import read
import xcquinox as xce
from functools import partial
from ase.units import Bohr
import os, optax, jax


CUDA backend failed to initialize: Found CUDA version 12010, but JAX was built against version 12030, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
def get_mol(atoms, basis='6-311++G**'):
    pos = atoms.positions
    spec = atoms.get_chemical_symbols()
    mol_input = [[s, p] for s, p in zip(spec, pos)]
    try:
        mol = gto.Mole(atom=mol_input, basis=atoms.info.get('basis',basis),spin=atoms.info.get('spin',0))
    except Exception:
        mol = gto.Mole(atom=mol_input, basis=atoms.info.get('basis','STO-3G'),spin=atoms.info.get('spin',0))
    return mol 

def get_rhos(rho, spin):
    rho0 = rho[0,0]
    drho = rho[0,1:4] + rho[1:4,0]
    tau = 0.5*(rho[1,1] + rho[2,2] + rho[3,3])

    if spin != 0:
        rho0_a = rho0[0]
        rho0_b = rho0[1]
        gamma_a, gamma_b = jnp.einsum('ij,ij->j',drho[:,0],drho[:,0]), jnp.einsum('ij,ij->j',drho[:,1],drho[:,1])              
        gamma_ab = jnp.einsum('ij,ij->j',drho[:,0],drho[:,1])
        tau_a, tau_b = tau
    else:
        rho0_a = rho0_b = rho0*0.5
        gamma_a=gamma_b=gamma_ab= jnp.einsum('ij,ij->j',drho[:],drho[:])*0.25
        tau_a = tau_b = tau*0.5
    return rho0_a, rho0_b, gamma_a, gamma_b, gamma_ab, tau_a, tau_b
    
def get_data_synth(xc_func, n=100):
    def get_rho(s, a):
        c0 = 2*(3*np.pi**2)**(1/3)
        c1 = 3/10*(3*np.pi**2)**(2/3)
        gamma = c0*s
        tau = c1*a+c0**2*s**2/8
        rho = np.zeros([len(a),6])
        rho[:, 1] = gamma
        rho[:,-1] = tau
        rho[:, 0] = 1
        return rho
    
    s_grid = jnp.concatenate([[0],jnp.exp(jnp.linspace(-10,4,n))])
    rho = []
    for s in s_grid:
        if 'MGGA' in xc_func:
            a_grid = jnp.concatenate([jnp.exp(jnp.linspace(jnp.log((s/100)+1e-8),8,n))])
        else:
            a_grid = jnp.array([0])
        rho.append(get_rho(s, a_grid))
        
    rho = jnp.concatenate(rho)
    
    fxc =  dft.numint.libxc.eval_xc(xc_func,rho.T, spin=0)[0]/dft.numint.libxc.eval_xc('LDA_X',rho.T, spin=0)[0] -1
 
    rho = jnp.asarray(rho)
    
    tdrho = xc.get_descriptors(rho[:,0]/2,rho[:,0]/2,(rho[:,1]/2)**2,(rho[:,1]/2)**2,(rho[:,1]/2)**2,rho[:,5]/2,rho[:,5]/2, spin_scaling=True)
    


    tFxc = torch.from_numpy(fxc)
    return tdrho[0], tFxc

def get_data(mol, xc_func ,full=False, enhance_spin=False, localnet=None):
    print('mol: ', mol.atom)
    try:
        mf = scf.UKS(mol)
    except:
        mf = dft.RKS(mol)
    mf.xc = 'PBE'
    mf.grids.level = 1
    mf.kernel()
    if not full:
        mf.grids.coords = coords
        mf.grids.weights = weights
    if localnet.spin_scaling:
        print('spin scaling, indicates exchange network')
        rho_alpha = mf._numint.eval_rho(mol, mf._numint.eval_ao(mol, mf.grids.coords, deriv=2) , mf.make_rdm1()[0], xctype='metaGGA',hermi=True)
        rho_beta = mf._numint.eval_rho(mol, mf._numint.eval_ao(mol, mf.grids.coords, deriv=2) , mf.make_rdm1()[1], xctype='metaGGA',hermi=True)
        fxc_a =  mf._numint.eval_xc(xc_func,(rho_alpha,rho_alpha*0), spin=1)[0]/mf._numint.eval_xc('LDA_X',(rho_alpha,rho_alpha*0), spin=1)[0] -1
        fxc_b =  mf._numint.eval_xc(xc_func,(rho_beta*0,rho_beta), spin=1)[0]/mf._numint.eval_xc('LDA_X',(rho_beta*0,rho_beta), spin=1)[0] -1
        print('fxc with xc_func = {} = {}'.format(fxc_a, xc_func))

        if mol.spin != 0 and sum(mol.nelec)>1:
            rho = jnp.concatenate([rho_alpha, rho_beta])
            fxc = jnp.concatenate([fxc_a, fxc_b])
        else:
            rho = rho_alpha
            fxc = fxc_a
    else:    
        print('no spin scaling, indicates correlation network')
        rho_alpha = mf._numint.eval_rho(mol, mf._numint.eval_ao(mol, mf.grids.coords, deriv=2) , mf.make_rdm1()[0], xctype='metaGGA',hermi=True)
        rho_beta = mf._numint.eval_rho(mol, mf._numint.eval_ao(mol, mf.grids.coords, deriv=2) , mf.make_rdm1()[1], xctype='metaGGA',hermi=True)
        exc = mf._numint.eval_xc(xc_func,(rho_alpha,rho_beta), spin=1)[0]
        print('exc with xc_func = {} = {}'.format(exc, xc_func))
        fxc = exc/mf._numint.eval_xc('LDA_C_PW',(rho_alpha, rho_beta), spin=1)[0] -1
#         fxc = exc
        rho = jnp.stack([rho_alpha,rho_beta], axis=-1)
    
    dm = jnp.array(mf.make_rdm1())
    ao_eval = jnp.array(mf._numint.eval_ao(mol, mf.grids.coords, deriv=1))
    rho = jnp.einsum('xij,yik,...jk->xy...i', ao_eval, ao_eval, dm)
    
    if dm.ndim == 3:
        rho_filt = (jnp.sum(rho[0,0],axis=0) > 1e-6)
    else:
        rho_filt = (rho[0,0] > 1e-6)
    tdrho = xc.get_descriptors(*get_rhos(rho, spin=1), spin_scaling=localnet.spin_scaling)
    
#     tdrho = torch.from_numpy(tdrho.detach().numpy().round(8))
        
    if localnet.spin_scaling:
        if mol.spin != 0 and sum(mol.nelec) > 1:
            tdrho = jnp.concatenate([tdrho[0],tdrho[1]])
            rho_filt = jnp.concatenate([rho_filt]*2)
            
        else:
            tdrho = tdrho[0]
    tdrho = tdrho[rho_filt]

    tFxc = jnp.array(fxc)[rho_filt]
#     tFxc = torch.from_snumpy(fxc)
    return tdrho, tFxc

In [4]:
PRETRAIN_LEVEL = 'MGGA'

TRAIN_NET = 'c'

REFERENCE_XC = 'PBE0'

N_HIDDEN = 16
DEPTH = 3
if PRETRAIN_LEVEL == 'GGA':
    if TRAIN_NET == 'x':
        localx = xce.net.eX(n_input=1, n_hidden=N_HIDDEN, use=[1], depth=DEPTH, lob=1.804)
    elif TRAIN_NET == 'c':
        localc = xce.net.eC(n_input=3, n_hidden=N_HIDDEN, use=[2], depth=DEPTH, ueg_limit=True)
elif PRETRAIN_LEVEL == 'MGGA':
    if TRAIN_NET == 'x':
        localx = xce.net.eX(n_input=2, n_hidden=N_HIDDEN, use=[1, 2], depth=DEPTH, ueg_limit=True, lob=1.174)
    elif TRAIN_NET == 'c':
        localc = xce.net.eC(n_input=4, n_hidden=N_HIDDEN, depth=DEPTH, use=[2,3], ueg_limit=True)

if TRAIN_NET == 'x':
    thislocal = localx
else:
    thislocal = localc
ueg = xce.xc.LDA_X()
xc = xce.xc.eXC(grid_models=[thislocal], heg_mult=True, level= {'GGA':2, 'MGGA':3, 'NONLOCAL':4}[PRETRAIN_LEVEL])

In [5]:
spins = {
    'Al': 1,
    'B' : 1,
    'Li': 1,
    'Na': 1,
    'Si': 2 ,
    'Be':0,
    'C': 2,
    'Cl': 1,
    'F': 1,
    'H': 1,
    'N': 3,
    'O': 2,
    'P': 3,
    'S': 2
}

selection = [2, 113, 25, 18, 11, 17, 114, 121, 101, 0, 20, 26, 29, 67, 28, 110, 125, 10, 115, 89, 105, 50]
atoms = [read('/home/awills/Documents/Research/ogdpyscf/dpyscf/data/haunschild_g2/g2_97.traj',':')[s] for s in selection]
ksr_atoms = atoms
if PRETRAIN_LEVEL=='MGGA':
    ksr_atoms = ksr_atoms[2:]
ksr_atoms = [Atoms('P',info={'spin':3}), Atoms('N', info={'spin':3}), Atoms('H', info={'spin':1}),Atoms('Li', info={'spin':1}), Atoms('O',info={'spin':2}),Atoms('Cl',info={'spin':1}),Atoms('Al',info={'spin':1}), Atoms('S',info={'spin':2})] + ksr_atoms
# ksr_atoms = [Atoms('H',info={'spin':1})]
mols = [get_mol(atoms) for atoms in ksr_atoms]
mols = [i for i in mols if len(i.atom) < 8]
for i in mols:
    print(i, i.atom, len(i.atom))

<pyscf.gto.mole.Mole object at 0x7451a04314b0> [['P', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7451a0432b60> [['N', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7451a04324d0> [['H', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7451a04322f0> [['Li', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7451a0432830> [['O', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7451a0433430> [['Cl', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7451a0431e40> [['Al', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7451a0433580> [['S', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7451a0433490> [['Li', array([ 0.      ,  0.      , -1.172697])], ['F', array([0.      , 0.      , 0.390899])]] 2
<pyscf.gto.mole.Mole object at 0x7451a0432cb0> [['C', array([ 0.      ,  0.      , -0.499686])], ['N', array([0.      , 0.      , 0.652056])], ['H', array([ 0.        ,  0.        , -1.56627401])]] 3
<pyscf.gto.mole.Mole object

In [6]:
thislocal.spin_scaling

False

In [7]:
# data = [get_data(mol, xc_func=ref, full=i<14) for i,mol in enumerate(mols)]
ref = 'PBE0'
data = [get_data(mol, xc_func=ref,full=True, localnet=thislocal) for i,mol in enumerate(mols)]
# 
# data = [get_data_synth(ref, 100)]
tdrho = jnp.concatenate([d[0] for d in data])
tFxc = jnp.concatenate([d[1] for d in data])

mol:  [['P', array([0., 0., 0.])]]


Initialize <pyscf.gto.mole.Mole object at 0x7451a04314b0> in UKS object of <class 'pyscf.dft.uks.UKS'>


converged SCF energy = -341.104145992717  <S^2> = 3.7502984  2S+1 = 4.0001492
no spin scaling, indicates correlation network
exc with xc_func = [-1.02664237e-02 -2.75034982e-03 -4.08130021e-03 ... -7.25655317e+00
 -7.25655317e+00 -7.25655317e+00] = PBE0
mol:  [['N', array([0., 0., 0.])]]


Initialize <pyscf.gto.mole.Mole object at 0x7451a0432b60> in UKS object of <class 'pyscf.dft.uks.UKS'>


converged SCF energy = -54.5289742046674  <S^2> = 3.7524945  2S+1 = 4.0012471
no spin scaling, indicates correlation network
exc with xc_func = [-5.96866425e-03 -2.51798665e-03 -5.83130694e-04 ... -3.33703592e+00
 -3.33703592e+00 -3.33703592e+00] = PBE0
mol:  [['H', array([0., 0., 0.])]]


...

converged SCF energy = -40.4598214864075  <S^2> = 3.170797e-13  2S+1 = 1
no spin scaling, indicates correlation network
exc with xc_func = [-0.00295239 -0.00121221 -0.00020696 ... -0.00027031 -0.00027031
 -0.00027031] = PBE0
mol:  [['C', array([0.      , 0.      , 0.179918])], ['H', array([ 0.      ,  0.855475, -0.539754])], ['H', array([ 0.      , -0.855475, -0.539754])]]


Initialize <pyscf.gto.mole.Mole object at 0x7451a04304f0> in UKS object of <class 'pyscf.dft.uks.UKS'>


converged SCF energy = -39.0756147483505  <S^2> = 6.2030381e-12  2S+1 = 1
no spin scaling, indicates correlation network
exc with xc_func = [-2.63869543e-03 -4.00286742e-04 -1.08589328e-03 ... -9.56249175e-01
 -9.56249175e-01 -9.56249175e-01] = PBE0
mol:  [['Si', array([0., 0., 0.])], ['H', array([0.855876, 0.855876, 0.855876])], ['H', array([-0.855876, -0.855876,  0.855876])], ['H', array([-0.855876,  0.855876, -0.855876])], ['H', array([ 0.855876, -0.855876, -0.855876])]]


Initialize <pyscf.gto.mole.Mole object at 0x7451a04313c0> in UKS object of <class 'pyscf.dft.uks.UKS'>


converged SCF energy = -291.719272437819  <S^2> = 1.3146817e-11  2S+1 = 1
no spin scaling, indicates correlation network
exc with xc_func = [-0.00288016 -0.00118216 -0.00020993 ... -0.0002218  -0.0002218
 -0.0002218 ] = PBE0


In [8]:
nan_filt = ~jnp.any((tdrho != tdrho),axis=-1)

tFxc = tFxc[nan_filt]
tdrho = tdrho[nan_filt,:]

In [9]:
tdrho[::].shape, tdrho[::]

((224145, 4),
 Array([[-3.55646847e+00,  2.20899060e-01,  2.04142939e+00,
          1.43859114e+00],
        [-4.04188739e+00,  2.26935426e-01,  2.65887948e+00,
          1.38496111e+00],
        [-3.20301698e+00,  2.08517375e-01,  1.63659032e+00,
          1.14131231e+00],
        ...,
        [-4.21483060e+00,  9.81437154e-14,  2.82063903e+00,
          5.40655126e-01],
        [-4.25518732e+00,  4.00346423e-13,  2.87882770e+00,
          2.76902720e-01],
        [-4.31527649e+00,  6.20392626e-13,  2.81798141e+00,
          1.35860915e+00]], dtype=float64))

In [14]:
cpus = jax.devices(backend='cpu')

In [16]:
PRINT_EVERY=200
scheduler = optax.exponential_decay(init_value = 1e-2, transition_begin=50, transition_steps=1000, decay_rate=0.9)
# optimizer = optax.adam(learning_rate = 1e-2)
optimizer = optax.adam(learning_rate = scheduler)

class PT_E_Loss():

    def __call__(self, model, inp, ref):

        pred = jax.vmap(model.net)(inp)[:, 0]

        err = pred-ref

        return jnp.mean(jnp.square(err))

trainer = xce.train.xcTrainer(model=thislocal, optim=optimizer, steps=100000, loss = PT_E_Loss(), do_jit=True)
if TRAIN_NET == 'x':
    inp = [tdrho[:, trainer.model.use]]
else:
    inp = [tdrho]
with jax.default_device(cpus[0]):
    newm = trainer(1, trainer.model, inp, [tFxc])
        

# for epoch in range(100000):
#     total_loss = 0
#     results = thislocal(tdrho[::])
#     loss = eloss(results, tFxc[::])
#     total_loss += loss.item()
#     loss.backward()

#     optimizer.step()
#     optimizer.zero_grad()
#     if epoch%PRINT_EVERY==0:
#         print('total loss {:.12f}'.format(total_loss))
        


Epoch 0
Epoch 0 :: Batch 0/1


CUDA backend failed to initialize: Found CUDA version 12010, but JAX was built against version 12030, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Batch Loss = 142.01953322295498
0, epoch_train_loss=142.01953322295498
Epoch 1
Epoch 1 :: Batch 0/1


CUDA backend failed to initialize: Found CUDA version 12010, but JAX was built against version 12030, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Batch Loss = 141.52423514100153
1, epoch_train_loss=141.52423514100153
Epoch 2
Epoch 2 :: Batch 0/1


...

CUDA backend failed to initialize: Found CUDA version 12010, but JAX was built against version 12030, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Batch Loss = 4.2646470377744095
79, epoch_train_loss=4.2646470377744095
Epoch 80
Epoch 80 :: Batch 0/1


CUDA backend failed to initialize: Found CUDA version 12010, but JAX was built against version 12030, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Batch Loss = 4.203499225050835
80, epoch_train_loss=4.203499225050835
Epoch 81
Epoch 81 :: Batch 0/1


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7452a39d66b0>>
Traceback (most recent call last):
  File "/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
jax.vmap(thislocal.net)(tdrho[:, [1,2]])[:, 0].shape

In [ ]:
jax.vmap(trainer.model.net)(tdrho[:, [1,2]])